In [1]:
import numpy as np
from sklearn.mixture import GaussianMixture

from sklearn import datasets

import torch

import pickle
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def make_gmm(n_comps, n_dims):
  section_size = n_dims // n_comps
  means = np.zeros((n_comps, n_dims))
  for i in range(n_comps):
    means[i][i*section_size:(i+1)*section_size] = 1
  data = np.zeros((10000,n_dims))
  for i in range(10000):
    data[i] = means[i%n_comps] + np.random.randn(n_dims)*.3
  gmm = GaussianMixture(n_comps)

  return gmm.fit(data)
# gmm = make_gmm(3,3)
# print(f'{np.round(gmm.means_,2)}')
# print(f'{np.round(gmm.weights_,2)}')
# print(f'{np.round(gmm.covariances_,2)}')

In [ ]:
# data = datasets.load_digits()['data']

# gmm = GaussianMixture(10, 'full', max_iter=50000)

# gmm = gmm.fit(data)

# samples = gmm.sample(10)


In [ ]:
# samples[0][0].shape
# for i in range(10):
#   plt.imshow(samples[0][i].reshape((8,8)), cmap='gray')
#   plt.title(samples[1][i])
#   plt.show()


In [ ]:
# for i in range(10):
#   digit = gmm.means_[i]
#   plt.imshow(digit.reshape((8,8)), cmap='gray')
#   plt.show()

In [ ]:
for size in [8,16,32,64]:
    for lr in ['1e-06', '1e-05', '0.0001']:
        with open(f'chkpt/mnist_gmm_rand_samples_{size}_{lr}.pickle','rb') as in_file:
            samples = pickle.load(in_file)['sample']
        for sample in samples[:3]:
            plt.imshow(sample[0], cmap='gray')
            plt.title(f'{size}_{lr}')
            plt.show()

In [ ]:
for size in [8,16,32,64]:
    for lr in ['1e-06', '1e-05', '0.0001']:
        with open(f'chkpt/mnist_gmm_rand_samples_{size}_{lr}.pickle','rb') as in_file:
            losses = np.array(pickle.load(in_file)['losses'])
        plt.plot(losses)
        plt.title(f'{size}_{lr}')
        plt.show()

In [2]:
from acflow import RealNVP
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

model = RealNVP(
        1, 5, 64, 10, (1,28,28), device, 10
      ).to(device)
  
# model.load_state_dict(torch.load('chkpt/mnist_gmm_rand_64_1e-06.tar')['model_state_dict'])
# sum(p.numel() for p in model.parameters())

In [3]:
def get_mnist(batch_size):
  transform = transforms.Compose(
    [
      transforms.ToTensor(),
      transforms.Normalize((0.1307,), (0.30811,))
    ]
  )

  train_set = tv.datasets.MNIST(
    '../data/', train=True, download=True, transform=transform
  )
  test_set = tv.datasets.MNIST(
    '../data/', train=False, download=True, transform=transform
  )

  train_loader = torch.utils.data.DataLoader(
    train_set, batch_size
  )
  test_loader = torch.utils.data.DataLoader(
    test_set, batch_size
  )

  return train_loader, test_loader

Parameter containing:
tensor([[0.5173, 0.8280, 0.3617,  ..., 0.4114, 0.9068, 0.2892],
        [0.3828, 0.1425, 0.9367,  ..., 0.0634, 0.4019, 0.1033],
        [0.4133, 0.1987, 0.6487,  ..., 0.2283, 0.6469, 0.6450],
        ...,
        [0.6439, 0.3167, 0.7326,  ..., 0.1028, 0.4715, 0.1172],
        [0.7797, 0.0696, 0.5914,  ..., 0.7257, 0.3238, 0.4277],
        [0.8165, 0.5931, 0.3967,  ..., 0.2942, 0.0739, 0.5887]],
       device='cuda:0', requires_grad=True)

In [ ]:
model.covs